# Regex in Pandas

**Inhalt:** Regular Expressions in Pandas anwenden

**Nötige Skills:** Regex in Python

**Lernziele:**
- Ein praktisches Beispiel kennenlernen, wo Regex nützlich sein kann

# Das Beispiel

Das Bundesamt für Statistik stellt oft Dateien in verknorkster Form zur Verfügung, zum Beispiel, wenn man Auswertungen nach Gemeinden, Bezirken und Kantonen über das interaktive Portal generieren lässt: https://www.pxweb.bfs.admin.ch/pxweb/de/

Das vorliegende Beispiel beinhaltet die Bevölkerungszahlen, gegliedert nach Zivilstand einerseits (ledig, verheiratet, etc) und nach räumlicher Struktur (Gemeinden, Kantone, etc) andererseits.

Das File ist gespeichert unter `dataprojects/BFS/px-x-0102010000_103.xlsx`

## Vorbereitung

In [1]:
import pandas as pd

## Datei laden

In [200]:
path = 'dataprojects/BFS/px-x-0102010000_103.xlsx'

In [216]:
df = pd.read_excel(path)

## Explorieren

In [217]:
df.head(20)

,Einheit,Bevölkerungstyp,Geschlecht,Zivilstand,Altersklasse - Total
0,Schweiz,Ständige Wohnbevölkerung,Geschlecht - Total,Ledig,3650651
1,Schweiz,Ständige Wohnbevölkerung,Geschlecht - Total,"Verheiratet, in eingetragener Partnerschaft",3583008
2,Schweiz,Ständige Wohnbevölkerung,Geschlecht - Total,"Verwitwet, durch Tod aufgelöste Partnerschaft",407408
3,Schweiz,Ständige Wohnbevölkerung,Geschlecht - Total,"Geschieden, unverheiratet, gerichtlich aufgelö...",685622
4,- Zürich,Ständige Wohnbevölkerung,Geschlecht - Total,Ledig,666873
5,- Zürich,Ständige Wohnbevölkerung,Geschlecht - Total,"Verheiratet, in eingetragener Partnerschaft",610396
6,- Zürich,Ständige Wohnbevölkerung,Geschlecht - Total,"Verwitwet, durch Tod aufgelöste Partnerschaft",63173
7,- Zürich,Ständige Wohnbevölkerung,Geschlecht - Total,"Geschieden, unverheiratet, gerichtlich aufgelö...",125889
8,>> Bezirk Affoltern,Ständige Wohnbevölkerung,Geschlecht - Total,Ledig,21785
9,>> Bezirk Affoltern,Ständige Wohnbevölkerung,Geschlecht - Total,"Verheiratet, in eingetragener Partnerschaft",23865


Offensichtlich müssen wir die Tabelle zuerst etwas umstellen, damit sie angenehm zu bearbeiten ist.

## Aufbereiten

Wir möchten:
- für jede Gemeinde, Bezirk eine Zeile
- für jeden Zivilstand eine Spalte

Die Lösung dafür kennen wir bereits: `df.pivot()`

In [218]:
df = df.pivot(index='Einheit', columns='Zivilstand', values='Altersklasse - Total')

In [219]:
df.head(2)

Zivilstand,"Geschieden, unverheiratet, gerichtlich aufgelöste Partnerschaft",Ledig,"Verheiratet, in eingetragener Partnerschaft","Verwitwet, durch Tod aufgelöste Partnerschaft"
Einheit,,,,
- Aargau,50715,276123,296940,29873
- Appenzell Ausserrhoden,4496,23055,24117,2875


Nun verschönern wir die Sache noch etwas...

In [220]:
df = df.reset_index()

In [221]:
df.columns.name = None

In [222]:
spalten = {
    'Geschieden, unverheiratet, gerichtlich aufgelöste Partnerschaft': 'Geschieden',
    'Verheiratet, in eingetragener Partnerschaft': 'Verheiratet',
    'Verwitwet, durch Tod aufgelöste Partnerschaft': 'Verwitwet'
}

In [223]:
df = df.rename(columns=spalten)

In [224]:
df.head(2)

,Einheit,Geschieden,Ledig,Verheiratet,Verwitwet
0,- Aargau,50715,276123,296940,29873
1,- Appenzell Ausserrhoden,4496,23055,24117,2875


## Die grographischen Einheiten...

Schauen wir uns mal näher an, was in der Spalte "Einheit" drinsteht

In [225]:
df['Einheit']

0                                           - Aargau
1                           - Appenzell Ausserrhoden
2                            - Appenzell Innerrhoden
3                                 - Basel-Landschaft
4                                      - Basel-Stadt
5                                     - Bern / Berne
6                              - Fribourg / Freiburg
7                                           - Genève
8                                           - Glarus
9                 - Graubünden / Grigioni / Grischun
10                                            - Jura
11                                          - Luzern
12                                       - Neuchâtel
13                                       - Nidwalden
14                                        - Obwalden
15                                    - Schaffhausen
16                                          - Schwyz
17                                       - Solothurn
18                                      - St. 

Welche Einheitstypen gibt es? Und welches Muster haben sie?
- Gemeinde ("...... 9999 Gemeindename")
- Bezirk (">> Bezirsname")
- Kantone ("- Kantonsname")
- Land ("Land")

Ziel:
- eine Spalte "Einheitstyp"
- eine Spalte "Einheitsnummer"
- eine Spalte "Einheitsname"

## Pandas-Funktionen, die Regex brauchen können

Einige Befehle heissen leicht anders, funktionieren aber sehr ähnlich wie in der re.Library

- **`str.contains(r"regex")`**: das Pendant zu `re.search()` - ja/nein-Antwort

- **`str.extract(r"regex")`**: auch ähnlich wie `re.search()` - Suchergebnis als Antwort

- **`str.replace(r"regex", "str")`**: das Pendant zu `re.sub()` - ersetzt Match mit String

Wir wenden diese Funktionen jetzt an.

### Aber zuerst ...

Zuerst brauchen wir die Regex-Ausdrücke, um die Einheiten zu erkennen. Am besten mit Tests beginnen, ob die Regex an einer Einheit anschlägt - und jeweils auch testen, ob die Regex bei Einheiten, die wir *nicht* wollen, auch *nicht* anschlägt.

In [226]:
# Test für Gemeinde
re.search(r"^\.{6}\d{4} .+$", "......0001 Aeugst am Albis")

<_sre.SRE_Match object; span=(0, 26), match='......0001 Aeugst am Albis'>

In [227]:
# Test für Bezirke
re.search(r"^>> .+$", ">> Wahlkreis Luzern-Stadt")

<_sre.SRE_Match object; span=(0, 25), match='>> Wahlkreis Luzern-Stadt'>

In [228]:
# Test für Kantone
re.search(r"^- .+$", "- Aargau")

<_sre.SRE_Match object; span=(0, 8), match='- Aargau'>

In [229]:
# Test für Land
re.search(r"Schweiz", "Schweiz")

<_sre.SRE_Match object; span=(0, 7), match='Schweiz'>

### Spalte "Einheitstyp"

Hier können wir die Funktion `str.contains()` gut brauchen.

Wir testen damit mal, ob die Einträge in der Spalte "Einheit" eine Gemeinde sind:

In [230]:
df['Einheit'].str.contains(r"^\.{6}\d{4} .+$")

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26       True
27       True
28       True
29       True
        ...  
2469    False
2470    False
2471    False
2472    False
2473    False
2474    False
2475    False
2476    False
2477    False
2478    False
2479    False
2480    False
2481    False
2482    False
2483    False
2484    False
2485    False
2486    False
2487    False
2488    False
2489    False
2490    False
2491    False
2492    False
2493    False
2494    False
2495    False
2496    False
2497    False
2498    False
Name: Einheit, Length: 2499, dtype: bool

Basierend auf dieser True/False-Liste können wir nun die Tabelle filtern und mit `df.loc[]` jeweils den richtigen Eintrag in unserer neuen Spalte "Einheitstyp" erzeugen.

In [231]:
df.loc[df['Einheit'].str.contains(r"^\.{6}\d{4} .+$"), 'Einheitstyp'] = "Gemeinde"
df.loc[df['Einheit'].str.contains(r"^>> .+$"), 'Einheitstyp'] = "Bezirk"
df.loc[df['Einheit'].str.contains(r"^- .+$"), 'Einheitstyp'] = "Kanton"
df.loc[df['Einheit'].str.contains(r"Schweiz"), 'Einheitstyp'] = "Land"

In [232]:
df

,Einheit,Geschieden,Ledig,Verheiratet,Verwitwet,Einheitstyp
0,- Aargau,50715,276123,296940,29873,Kanton
1,- Appenzell Ausserrhoden,4496,23055,24117,2875,Kanton
2,- Appenzell Innerrhoden,814,7338,6971,851,Kanton
3,- Basel-Landschaft,22994,113105,131580,15539,Kanton
4,- Basel-Stadt,18693,89611,72481,11023,Kanton
5,- Bern / Berne,84364,437186,439372,56513,Kanton
6,- Fribourg / Freiburg,23511,138940,131592,13403,Kanton
7,- Genève,46072,224586,193557,20496,Kanton
8,- Glarus,3066,16607,18015,2338,Kanton
9,- Graubünden / Grigioni / Grischun,14957,83298,87623,10724,Kanton


### Spalte "Einheitsnummer"

Hier kommt die Funktion `str.extract()` gelegen.

In [233]:
df['Einheit'].str.extract(r"^\.{6}(\d{4}) .+$")

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


Wir können den Output dieser Funktion für die neue Spalte setzen.

In [234]:
df['Einheitsnummer'] = df['Einheit'].str.extract(r"^\.{6}(\d{4}) .+$")

In [235]:
df.head(40)

,Einheit,Geschieden,Ledig,Verheiratet,Verwitwet,Einheitstyp,Einheitsnummer
0,- Aargau,50715,276123,296940,29873,Kanton,NaN
1,- Appenzell Ausserrhoden,4496,23055,24117,2875,Kanton,NaN
2,- Appenzell Innerrhoden,814,7338,6971,851,Kanton,NaN
3,- Basel-Landschaft,22994,113105,131580,15539,Kanton,NaN
4,- Basel-Stadt,18693,89611,72481,11023,Kanton,NaN
5,- Bern / Berne,84364,437186,439372,56513,Kanton,NaN
6,- Fribourg / Freiburg,23511,138940,131592,13403,Kanton,NaN
7,- Genève,46072,224586,193557,20496,Kanton,NaN
8,- Glarus,3066,16607,18015,2338,Kanton,NaN
9,- Graubünden / Grigioni / Grischun,14957,83298,87623,10724,Kanton,NaN


**Achtung: ** Dieser Trick funktioniert hier, weil die Kantone und Bezirke *keine* Nummer haben - wir schreiben dort jetzt einfach "NaN" hin. Würden wir dieselbe Regex-Extraktion auch auf Kantone anwenden, müssten wir aufpassen, dass wir die Werte der Gemeinden, die dann ebenfalls "NaN" wären, nicht wieder überschreiben würden.

### Spalte "Einheitsname"

Es gibt zig Varianten, wie wir hier zum Ziel kommen können. Wir wählen mal den folgenden:
- Spalte "Einheit" kopieren
- Jeweils für Gemeinden, Bezirke, Kantone separat den ganzen Käse am Anfang rauslöschen, der nicht zum Namen gehört.

In [236]:
df['Einheitsname'] = df['Einheit']

In [237]:
df.head(2)

,Einheit,Geschieden,Ledig,Verheiratet,Verwitwet,Einheitstyp,Einheitsnummer,Einheitsname
0,- Aargau,50715,276123,296940,29873,Kanton,NaN,- Aargau
1,- Appenzell Ausserrhoden,4496,23055,24117,2875,Kanton,NaN,- Appenzell Ausserrhoden


In [238]:
# Gemeinden
df['Einheitsname'].str.replace(r"^\.{6}\d{4} ", "")

0                                           - Aargau
1                           - Appenzell Ausserrhoden
2                            - Appenzell Innerrhoden
3                                 - Basel-Landschaft
4                                      - Basel-Stadt
5                                     - Bern / Berne
6                              - Fribourg / Freiburg
7                                           - Genève
8                                           - Glarus
9                 - Graubünden / Grigioni / Grischun
10                                            - Jura
11                                          - Luzern
12                                       - Neuchâtel
13                                       - Nidwalden
14                                        - Obwalden
15                                    - Schaffhausen
16                                          - Schwyz
17                                       - Solothurn
18                                      - St. 

In [239]:
# Gemeinden
df['Einheitsname'] = df['Einheitsname'].str.replace(r"^\.{6}\d{4} ", "")

In [240]:
# Bezirke
df['Einheitsname'] = df['Einheitsname'].str.replace(r"^>> ", "")

In [241]:
# Kantone
df['Einheitsname'] = df['Einheitsname'].str.replace(r"^- ", "")

In [242]:
df

,Einheit,Geschieden,Ledig,Verheiratet,Verwitwet,Einheitstyp,Einheitsnummer,Einheitsname
0,- Aargau,50715,276123,296940,29873,Kanton,NaN,Aargau
1,- Appenzell Ausserrhoden,4496,23055,24117,2875,Kanton,NaN,Appenzell Ausserrhoden
2,- Appenzell Innerrhoden,814,7338,6971,851,Kanton,NaN,Appenzell Innerrhoden
3,- Basel-Landschaft,22994,113105,131580,15539,Kanton,NaN,Basel-Landschaft
4,- Basel-Stadt,18693,89611,72481,11023,Kanton,NaN,Basel-Stadt
5,- Bern / Berne,84364,437186,439372,56513,Kanton,NaN,Bern / Berne
6,- Fribourg / Freiburg,23511,138940,131592,13403,Kanton,NaN,Fribourg / Freiburg
7,- Genève,46072,224586,193557,20496,Kanton,NaN,Genève
8,- Glarus,3066,16607,18015,2338,Kanton,NaN,Glarus
9,- Graubünden / Grigioni / Grischun,14957,83298,87623,10724,Kanton,NaN,Graubünden / Grigioni / Grischun
